In [7]:
import os

%pwd

# we need to jump back to the root folder 

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\NLP\\End To End Text Summarizer appl\\TextSummarizer'

In [2]:
#os.chdir('c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\NLP\\End To End Text Summarizer appl\\TextSummarizer')

os.chdir('../')
%pwd

'c:\\Users\\sredekar\\OneDrive - Interpublic\\Desktop\\Datascience\\NLP\\End To End Text Summarizer appl\\TextSummarizer'

In [8]:
#3. Update entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [9]:
#4. Update the configuration manager in src config
from TextSummarizer.constants import * # this will read from constant
from TextSummarizer.utils.common import read_yaml, create_directories # this will read from utlis


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [10]:
#5. update the components
import os
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [11]:
# 5. update the pipeline
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-03-29 18:00:39,033: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-29 18:00:39,033: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-29 18:00:39,033: INFO: common: created directory at: artifacts]


AttributeError: 'function' object has no attribute 'tokenizer_name'